<a href="https://colab.research.google.com/github/Shanmukhpappala/D-3F-308/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [7]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [8]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-8-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [9]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [10]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [11]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [12]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [13]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [14]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [15]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [16]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [17]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [18]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

1). Here's a summary of how the accuracy of the validation set is affected by increasing or reducing the percentage of the validation set:

**Increasing the Percentage of Validation Set:**

- Pros:
  - Provides a more reliable estimate of model generalization.
  - Helps identify overfitting earlier.
- Cons:
  - Reduces the amount of data available for training.
  - Increases computational cost.

**Reducing the Percentage of Validation Set:**

- Pros:
  - Provides more data for training.
  - May lead to faster training and lower computational requirements.
- Cons:
  - Can result in less reliable performance estimates.
  - Makes early detection of overfitting challenging.

The choice of the validation set percentage should consider the trade-off between reliable evaluation and the need for an adequate amount of training data. Techniques like k-fold cross-validation can be used to strike a balance between these factors effectively.

2). The size of the training and validation sets affects how well we can predict the accuracy on the test set using the validation set.

- Large Train and Validation Sets: These are likely to be representative and provide reliable performance estimates. Validation accuracy is a good indicator of test set performance.

- Small Validation Set with a Large Training Set: While the training set can capture data patterns, a small validation set may lead to less reliable estimates. Consider cross-validation for robust assessments.

- Small Train Set with a Small Validation Set: Limited data can hinder model learning, and performance estimates from a small validation set may be unreliable. A separate, unseen test set is crucial for accurate generalization assessment.

In summary, the size of these sets impacts the reliability of performance estimates. Larger, representative sets offer more accurate predictions, while limited data may require cross-validation and a separate test set for comprehensive evaluation.

3). Certainly, here are the key points for choosing the percentage to reserve for the validation set while balancing factors:

- Common practice is to use an 80-20 or 70-30 train-validation split for larger datasets.
- Smaller validation percentages (e.g., 10-20%) are suitable for larger datasets.
- Consider k-fold cross-validation for small datasets or more robust estimates (commonly k=5 or k=10).
- Use stratified sampling when dealing with imbalanced classes to maintain a representative distribution.
- Adjust the split percentage based on data availability and specific goals.
- Experimentation and evaluation of model performance with different splits can help find the right balance between training and validation data.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [19]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [20]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1)Certainly! Here's a summary of the key points:

- Averaging validation accuracy across multiple splits, such as through k-fold cross-validation, provides more consistent and reliable results compared to a single train-test split.
- This approach reduces the variance in evaluation metrics, making them less dependent on the specific data points in the test set.
- It leads to better generalization estimates, helping you assess how well your model is likely to perform on unseen data.
- Averaging over multiple splits ensures unbiased evaluation and is particularly useful for model selection and hyperparameter tuning.
- Overall, using cross-validation techniques enhances the robustness and trustworthiness of your machine learning model evaluations.

2)Certainly, here's a concise summary:

- Averaging validation accuracy through techniques like k-fold cross-validation provides a more accurate estimate of a model's generalization performance compared to a single train-test split.
- Cross-validation reduces variance, offers better representation of model performance, and improves model assessment by evaluating performance across different data subsets.
- However, it estimates how well your model performs within your dataset, not its actual test accuracy on a completely new and independent dataset.
- For real-world performance assessment, it's essential to use a separate, unseen test dataset. Cross-validation aids in model development and selection, enhancing confidence in your model's generalization ability.

3)Here's a concise summary of the effect of the number of iterations on model estimation:

- The number of iterations (epochs) in training a machine learning model has a significant impact on its performance estimate.
- Too few iterations can lead to underfitting, where the model doesn't capture underlying patterns.
- Too many iterations can lead to overfitting, where the model fits noise in the training data.
- Monitoring validation metrics helps identify the point at which further iterations stop improving generalization.
- Early stopping is a technique to find the optimal number of iterations by halting training when performance starts degrading.
- The choice of the number of iterations involves a trade-off between computational resources and model performance.

4) Certainly, here's a summary of dealing with very small training or validation datasets by increasing iterations:

- Increasing iterations can help mitigate underfitting in small training datasets by allowing the model to learn better representations through repeated exposure to limited data.

- However, increasing iterations without caution may lead to overfitting, especially with small datasets. Regularization techniques and careful monitoring of validation metrics are essential to prevent overfitting.

- When dealing with a very small validation dataset, it can be challenging to assess model performance reliably. Techniques like k-fold cross-validation or bootstrapping can provide more stable performance estimates.

- Data augmentation methods can effectively increase the effective size of small training datasets by generating additional training samples through transformations.

- Consider transfer learning, especially when you have a small dataset for a specific task. Pre-trained models can be fine-tuned on your data, leveraging knowledge from larger datasets.

In summary, increasing iterations is a useful strategy for small datasets, but it should be used in combination with regularization, data augmentation, appropriate model selection, and, if applicable, transfer learning to address the limitations of limited data.4)